# Prova 2 - Econometria 1

## Nomes: Hugo Trigueiro e Kevin Mattos

In [90]:
import wooldridge as wd
import pandas as pd
import numpy as np

In [35]:
pd.set_option("display.max_columns", None)

In [36]:
jtrain = wd.data('jtrain')

In [37]:
df = jtrain.copy()

In [38]:
df.head()

,year,fcode,employ,sales,avgsal,scrap,rework,tothrs,union,grant,d89,d88,totrain,hrsemp,lscrap,lemploy,lsales,lrework,lhrsemp,lscrap_1,grant_1,clscrap,cgrant,clemploy,clsales,lavgsal,clavgsal,cgrant_1,chrsemp,clhrsemp
0,1987,410032.0,100.0,47000000.0,35000.0,NaN,NaN,12.0,0,0,0,0,100.0,12.000000,NaN,4.605170,17.665659,NaN,2.564949,NaN,0,NaN,0,NaN,NaN,10.463103,NaN,NaN,NaN,NaN
1,1988,410032.0,131.0,43000000.0,37000.0,NaN,NaN,8.0,0,0,0,1,50.0,3.053435,NaN,4.875197,17.576710,NaN,1.399565,NaN,0,NaN,0,0.270027,-0.088949,10.518673,0.055570,0.0,-8.946565,-1.165385
2,1989,410032.0,123.0,49000000.0,39000.0,NaN,NaN,8.0,0,0,1,0,50.0,3.252033,NaN,4.812184,17.707331,NaN,1.447397,NaN,0,NaN,0,-0.063013,0.130621,10.571317,0.052644,0.0,0.198597,0.047832
3,1987,410440.0,12.0,1560000.0,10500.0,NaN,NaN,12.0,0,0,0,0,12.0,12.000000,NaN,2.484907,14.260197,NaN,2.564949,NaN,0,NaN,0,NaN,NaN,9.259130,NaN,NaN,NaN,NaN
4,1988,410440.0,13.0,1970000.0,11000.0,NaN,NaN,12.0,0,0,0,1,13.0,12.000000,NaN,2.564949,14.493544,NaN,2.564949,NaN,0,NaN,0,0.080043,0.233347,9.305651,0.046520,0.0,0.000000,0.000000


In [39]:
df["firm_sub_1988"] = (df["year"] == 1988) & (df["grant"] == 1).astype(int)
df["firm_sub_1989"] = (df["year"] == 1989) & (df["grant"] == 1).astype(int)
df["firm_sub_not"] = (df["grant"] == 0).astype(int)

In [40]:
df.head()

,year,fcode,employ,sales,avgsal,scrap,rework,tothrs,union,grant,d89,d88,totrain,hrsemp,lscrap,lemploy,lsales,lrework,lhrsemp,lscrap_1,grant_1,clscrap,cgrant,clemploy,clsales,lavgsal,clavgsal,cgrant_1,chrsemp,clhrsemp,firm_sub_1988,firm_sub_1989,firm_sub_not
0,1987,410032.0,100.0,47000000.0,35000.0,NaN,NaN,12.0,0,0,0,0,100.0,12.000000,NaN,4.605170,17.665659,NaN,2.564949,NaN,0,NaN,0,NaN,NaN,10.463103,NaN,NaN,NaN,NaN,False,False,1
1,1988,410032.0,131.0,43000000.0,37000.0,NaN,NaN,8.0,0,0,0,1,50.0,3.053435,NaN,4.875197,17.576710,NaN,1.399565,NaN,0,NaN,0,0.270027,-0.088949,10.518673,0.055570,0.0,-8.946565,-1.165385,False,False,1
2,1989,410032.0,123.0,49000000.0,39000.0,NaN,NaN,8.0,0,0,1,0,50.0,3.252033,NaN,4.812184,17.707331,NaN,1.447397,NaN,0,NaN,0,-0.063013,0.130621,10.571317,0.052644,0.0,0.198597,0.047832,False,False,1
3,1987,410440.0,12.0,1560000.0,10500.0,NaN,NaN,12.0,0,0,0,0,12.0,12.000000,NaN,2.484907,14.260197,NaN,2.564949,NaN,0,NaN,0,NaN,NaN,9.259130,NaN,NaN,NaN,NaN,False,False,1
4,1988,410440.0,13.0,1970000.0,11000.0,NaN,NaN,12.0,0,0,0,1,13.0,12.000000,NaN,2.564949,14.493544,NaN,2.564949,NaN,0,NaN,0,0.080043,0.233347,9.305651,0.046520,0.0,0.000000,0.000000,False,False,1


In [41]:
df = pd.concat([df.set_index("fcode"), df.groupby(["fcode"])["firm_sub_1988"].max().astype(int).rename("grant_1988")], axis=1).reset_index()
df = pd.concat([df.set_index("fcode"), df.groupby(["fcode"])["firm_sub_1989"].max().astype(int).rename("grant_1989")], axis=1).reset_index()
df = pd.concat([df.set_index("fcode"), (df.groupby(["fcode"])["firm_sub_not"].sum() == 3).astype(int).astype(int).rename("grant_no")], axis=1).reset_index()

In [42]:
df88 = df.groupby(["year", "grant_1988"]).agg({"fcode": "count", "hrsemp": ["mean", "std"]}).reset_index(level=1)
df88 = df88.loc[df88.grant_1988 == 1].drop(["grant_1988"], level=0, axis=1).reset_index()
df88.columns = ["year", "count", "mean", "std"]
df88["descricao"] = "Firmas que receberam subsídio em 1988"
df88 = df88.groupby(["descricao", "year"]).mean()
df88

count       mean        std
descricao                             year                             
Firmas que receberam subsídio em 1988 1987     36   7.672105  19.574833
                                      1988     36  35.978342  36.956019
                                      1989     36  10.055916  19.474582

In [43]:
df89 = df.groupby(["year", "grant_1989"]).agg({"fcode": "count", "hrsemp": ["mean", "std"]}).reset_index(level=1)
df89 = df89.loc[df89.grant_1989 == 1].drop(["grant_1989"], level=0, axis=1).reset_index()
df89.columns = ["year", "count", "mean", "std"]
df89["descricao"] = "Firmas que receberam subsídio em 1989"
df89 = df89.groupby(["descricao", "year"]).mean()
df89

count       mean        std
descricao                             year                             
Firmas que receberam subsídio em 1989 1987     30   6.124542  11.309335
                                      1988     30   9.315550  17.360545
                                      1989     30  50.649975  36.217713

In [44]:
dfnot = df.groupby(["year", "grant_no"]).agg({"fcode": "count", "hrsemp": ["mean", "std"]}).reset_index(level=1)
dfnot = dfnot.loc[dfnot.grant_no == 1].drop(["grant_no"], level=0, axis=1).reset_index()
dfnot.columns = ["year", "count", "mean", "std"]
dfnot["descricao"] = "Firmas que não receberam subsídio"
dfnot = dfnot.groupby(["descricao", "year"]).mean()
dfnot

count       mean        std
descricao                         year                             
Firmas que não receberam subsídio 1987     91  10.702936  18.314595
                                  1988     91   9.817479  18.627084
                                  1989     91  12.352485  23.777301

In [45]:
df2 = pd.concat([df88, df89, dfnot])
df2

count       mean        std
descricao                             year                             
Firmas que receberam subsídio em 1988 1987     36   7.672105  19.574833
                                      1988     36  35.978342  36.956019
                                      1989     36  10.055916  19.474582
Firmas que receberam subsídio em 1989 1987     30   6.124542  11.309335
                                      1988     30   9.315550  17.360545
                                      1989     30  50.649975  36.217713
Firmas que não receberam subsídio     1987     91  10.702936  18.314595
                                      1988     91   9.817479  18.627084
                                      1989     91  12.352485  23.777301

### 1) Estatísticas Descritivas

In [46]:
df2

count       mean        std
descricao                             year                             
Firmas que receberam subsídio em 1988 1987     36   7.672105  19.574833
                                      1988     36  35.978342  36.956019
                                      1989     36  10.055916  19.474582
Firmas que receberam subsídio em 1989 1987     30   6.124542  11.309335
                                      1988     30   9.315550  17.360545
                                      1989     30  50.649975  36.217713
Firmas que não receberam subsídio     1987     91  10.702936  18.314595
                                      1988     91   9.817479  18.627084
                                      1989     91  12.352485  23.777301

In [118]:
def round2(x):
    return round(x, 2)

In [122]:
jtrain.describe().T.apply(round2).to_csv("teste.csv")

### 2) Equação Estimada e Hipóteses para o modelo escolhido

In [51]:
import statsmodels.formula.api as smf

$$
\LARGE hrsemp = \beta_0 + \beta_1 grant 
$$

In [52]:
formula = "hrsemp ~ grant + cgrant_1"

In [55]:
reg = smf.ols(formula, data=jtrain).fit()

In [57]:
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:                 hrsemp   R-squared:                       0.225
Model:                            OLS   Adj. R-squared:                  0.219
Method:                 Least Squares   F-statistic:                     37.46
Date:                Wed, 28 Apr 2021   Prob (F-statistic):           5.21e-15
Time:                        22:15:20   Log-Likelihood:                -1210.9
No. Observations:                 261   AIC:                             2428.
Df Residuals:                     258   BIC:                             2439.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.8111      1.949      5.546      0.0

### 3) Coeficientes, Desvio-padrão, $R^2$

#### 3.1) Coeficientes:

In [58]:
reg.params

Intercept    10.811080
grant        32.130070
cgrant_1     -0.755164
dtype: float64

#### 3.2) Desvio-padrão:

In [106]:
# Erro padrão
reg.bse

Intercept    1.949250
grant        3.815012
cgrant_1     4.682840
dtype: float64

$$
std_{err} = \frac{std}{\sqrt{n}} \rightarrow std_{err} \sqrt{n} = std 
$$

In [91]:
reg.bse*np.sqrt(reg.nobs)

Intercept    31.491106
grant        61.633399
cgrant_1     75.653595
dtype: float64

#### 3.3) $R^2$:

In [109]:
reg.rsquared

0.22504606457822385

### 4) Teste de hipótese dos coeficientes:

#### 4.1) Teste t

In [164]:
def hypos(reg, value_hypo="0"):
    params = reg.params.index
    hypothesis = [param + f" = {value_hypo}" for n, param in enumerate(reg.params.index)]
    hypothesis = ", ".join(hypothesis)
    return hypothesis

In [170]:
reg.t_test(hypos(reg))

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0            10.8111      1.949      5.546      0.000       6.973      14.650
c1            32.1301      3.815      8.422      0.000      24.618      39.643
c2            -0.7552      4.683     -0.161      0.872      -9.977       8.466

#### 4.2) Teste F

In [171]:
reg.f_test(hypos(reg))

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[69.26430528]]), p=6.884114885249228e-33, df_denom=258, df_num=3>

#### 4.3) Teste LM

In [173]:
from statsmodels.stats.diagnostic import linear_lm

In [ ]:
linear_lm()

### 5) Discussão dos resultados encontrados